<a href="https://colab.research.google.com/github/mahault/eduplate/blob/main/Eduplate_toy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic toy example - full inference loop and explanation

In [ ]:
!pip install pgmpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

1. Define the Bayesian Network
Based on the provided graph, the variables are:



*   Active Ing (Active Ingredient)
* Food
* Quantity
* Effectiveness
* Types of Effects

We will use the pgmpy library to define the Bayesian Network.



In [ ]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

# Define the structure of the Bayesian Network
model = BayesianNetwork([('ActiveIng', 'Effectiveness'),
                         ('Food', 'Effectiveness'),
                         ('Quantity', 'Effectiveness'),
                         ('Effectiveness', 'TypesOfEffects')])

# Define the CPDs (using dummy probabilities for illustration)
cpd_active_ing = TabularCPD(variable='ActiveIng', variable_card=2, values=[[0.7], [0.3]])
cpd_food = TabularCPD(variable='Food', variable_card=2, values=[[0.6], [0.4]])
cpd_quantity = TabularCPD(variable='Quantity', variable_card=2, values=[[0.5], [0.5]])

cpd_effectiveness = TabularCPD(variable='Effectiveness', variable_card=2,
                               values=[[0.95, 0.8, 0.8, 0.6, 0.8, 0.6, 0.6, 0.2],
                                       [0.05, 0.2, 0.2, 0.4, 0.2, 0.4, 0.4, 0.8]],
                               evidence=['ActiveIng', 'Food', 'Quantity'],
                               evidence_card=[2, 2, 2])

cpd_types_of_effects = TabularCPD(variable='TypesOfEffects', variable_card=2,
                                  values=[[0.9, 0.8],  # P(TypesOfEffects=0 | Effectiveness=0) and P(TypesOfEffects=0 | Effectiveness=1)
                                          [0.1, 0.2]], # P(TypesOfEffects=1 | Effectiveness=0) and P(TypesOfEffects=1 | Effectiveness=1)
                                  evidence=['Effectiveness'],
                                  evidence_card=[2])

# Add the CPDs to the model
model.add_cpds(cpd_active_ing, cpd_food, cpd_quantity, cpd_effectiveness, cpd_types_of_effects)

# Check if the model is valid
assert model.check_model()

# Define the inference object
inference = VariableElimination(model)

2. Data Preprocessing

Extract relevant facts from the transcript. For simplicity, let's assume we have a dictionary of extracted facts.

In [ ]:
# Example observations extracted from the transcript
observations = {'ActiveIng': 1, 'Food': 0, 'Quantity': 1}


3. Perform Inference and Generate Explanations

Perform inference to compute the probability of Effectiveness and TypesOfEffects.

In [ ]:
# Perform inference to find the probability of Effectiveness
prob_effectiveness = inference.query(variables=['Effectiveness'], evidence=observations)
print(f"Probability of Effectiveness: {prob_effectiveness}")

# Perform inference to find the probability of Types of Effects
prob_types_of_effects = inference.query(variables=['TypesOfEffects'], evidence=observations)
print(f"Probability of Types of Effects: {prob_types_of_effects}")

# Generate explanations
explanation_effectiveness = f"Given the Active Ingredient, Food, and Quantity, the probability of Effectiveness being high is {prob_effectiveness.values[1]:.2f}."
explanation_types_of_effects = f"Given the Effectiveness, the probability of having significant Types of Effects is {prob_types_of_effects.values[1]:.2f}."

print(f"Explanation for Effectiveness: {explanation_effectiveness}")
print(f"Explanation for Types of Effects: {explanation_types_of_effects}")


Probability of Effectiveness: +------------------+----------------------+
| Effectiveness    |   phi(Effectiveness) |
+==================+======================+
| Effectiveness(0) |               0.6000 |
+------------------+----------------------+
| Effectiveness(1) |               0.4000 |
+------------------+----------------------+
Probability of Types of Effects: +-------------------+-----------------------+
| TypesOfEffects    |   phi(TypesOfEffects) |
+===================+=======================+
| TypesOfEffects(0) |                0.8600 |
+-------------------+-----------------------+
| TypesOfEffects(1) |                0.1400 |
+-------------------+-----------------------+
Explanation for Effectiveness: Given the Active Ingredient, Food, and Quantity, the probability of Effectiveness being high is 0.40.
Explanation for Types of Effects: Given the Effectiveness, the probability of having significant Types of Effects is 0.14.


# Now let's say you want to learn the values of the probabilities rather than prespecify them

1. Define the Network Structure

First, define the network structure as done previously.

In [89]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

# Define the structure of the Bayesian Network
model = BayesianNetwork([('ActiveIng', 'Effectiveness'),
                         ('Food', 'Effectiveness'),
                         ('Quantity', 'Effectiveness'),
                         ('Effectiveness', 'TypesOfEffects'),
                         ('ActiveIng', 'TypesOfEffects'),
                         ('Food', 'TypesOfEffects')])

2. Prepare the Data

Assume you have a dataset in a CSV file. Load this data into a pandas DataFrame.


In [90]:
import pandas as pd

# Load data
# data = pd.read_csv('your_data.csv')

# Sample data
data = pd.DataFrame({
    'ActiveIng': [1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1],
    'Food': [0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0],
    'Quantity': [1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1],
    'Effectiveness': [1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1],
    'TypesOfEffects': [0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0]
})


3. Parameter Learning

Use the MaximumLikelihoodEstimator to learn the CPDs from the data.

In [91]:
from pgmpy.estimators import MaximumLikelihoodEstimator


# Fit the model using MaximumLikelihoodEstimator
model.fit(data, estimator=MaximumLikelihoodEstimator)

4. Perform Inference

After fitting the model, perform inference as shown previously.

In [92]:
from pgmpy.inference import VariableElimination

# Define the inference object
inference = VariableElimination(model)

# Example observations extracted from the transcript
observations = {'ActiveIng': 1, 'Food': 0, 'Quantity': 1}

# Perform inference to find the probability of Effectiveness
prob_effectiveness = inference.query(variables=['Effectiveness'], evidence=observations)
print(f"Probability of Effectiveness: {prob_effectiveness}")

# Perform inference to find the probability of Types of Effects
prob_types_of_effects = inference.query(variables=['TypesOfEffects'], evidence=observations)
print(f"Probability of Types of Effects: {prob_types_of_effects}")

# Generate explanations
explanation_effectiveness = f"Given the Active Ingredient, Food, and Quantity, the probability of Effectiveness being high is {prob_effectiveness.values[1]:.2f}."
explanation_types_of_effects = f"Given the Effectiveness, the Active Ingredient, and the Food, the probability of having significant Types of Effects is {prob_types_of_effects.values[1]:.2f}."

print(f"Explanation for Effectiveness: {explanation_effectiveness}")
print(f"Explanation for Types of Effects: {explanation_types_of_effects}")

Probability of Effectiveness: +------------------+----------------------+
| Effectiveness    |   phi(Effectiveness) |
+==================+======================+
| Effectiveness(0) |               0.0000 |
+------------------+----------------------+
| Effectiveness(1) |               1.0000 |
+------------------+----------------------+
Probability of Types of Effects: +-------------------+-----------------------+
| TypesOfEffects    |   phi(TypesOfEffects) |
+===================+=======================+
| TypesOfEffects(0) |                1.0000 |
+-------------------+-----------------------+
| TypesOfEffects(1) |                0.0000 |
+-------------------+-----------------------+
Explanation for Effectiveness: Given the Active Ingredient, Food, and Quantity, the probability of Effectiveness being high is 1.00.
Explanation for Types of Effects: Given the Effectiveness, the Active Ingredient, and the Food, the probability of having significant Types of Effects is 0.00.


# Now onto the text parsing aspect

Text Preprocessing:

* Tokenization
* Named Entity Recognition (NER)
* Part-of-Speech (POS) Tagging

## Spacy

In [ ]:
# Install spaCy and Download a Model

!pip install spacy
!python -m spacy download en_core_web_lg


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 459.8 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy
import pandas as pd
import re

# Load spaCy model
nlp = spacy.load("en_core_web_lg")

def categorize_text(text):
    doc = nlp(text)

    # Initialize categories
    food = None
    quantity = None
    effectiveness = None
    types_of_effects = None

    # Iterate over tokens in the document
    for token in doc:
        # Check for food category
        if token.text.lower() in ["meat", "veggie"]:
            food = 0 if token.text.lower() == "meat" else 1

        # Check for quantity category
        if token.text.lower() in ["little", "lot"]:
            quantity = 0 if token.text.lower() == "little" else 1

        # Check for effectiveness category
        if token.text.lower() in ["low", "high"]:
            effectiveness = 0 if token.text.lower() == "low" else 1

        # Check for types of effects category
        if token.text.lower() in ["negative", "positive"]:
            types_of_effects = 0 if token.text.lower() == "negative" else 1

    return food, quantity, effectiveness, types_of_effects


In [ ]:
text1 = "The meal contained a lot of meat and had a high effectiveness with positive effects."
text2 = "The veggie dish was served in little quantity and had a low effectiveness with negative effects."

food1, quantity1, effectiveness1, types_of_effects1 = categorize_text(text1)
food2, quantity2, effectiveness2, types_of_effects2 = categorize_text(text2)

print(f"Text 1: Food={food1}, Quantity={quantity1}, Effectiveness={effectiveness1}, Types of Effects={types_of_effects1}")
print(f"Text 2: Food={food2}, Quantity={quantity2}, Effectiveness={effectiveness2}, Types of Effects={types_of_effects2}")

Text 1: Food=0, Quantity=1, Effectiveness=1, Types of Effects=1
Text 2: Food=1, Quantity=0, Effectiveness=0, Types of Effects=0


As you can see it's working ok, but it's not super flexible

## Let's use similarity scores

You can use Hugging Face Transformers for state-of-the-art NLP models that can handle named entity recognition (NER), relation extraction, and more.
Stanford CoreNLP provides various NLP tools including NER, dependency parsing, and relation extraction.
AllenNLP is built on PyTorch, it offers extensive tools for NLP including pre-trained models for NER and relation extraction.

In [ ]:
import spacy

nlp = spacy.load("en_core_web_lg")

# Define category vectors
food_vectors = {
    "meat": nlp("meat").vector,
    "veggie": nlp("vegetable").vector
}

quantity_vectors = {
    "little": nlp("small").vector,
    "lot": nlp("large").vector
}

effectiveness_vectors = {
    "low": nlp("low").vector,
    "high": nlp("high").vector
}

types_of_effects_vectors = {
    "negative": nlp("negative").vector,
    "positive": nlp("positive").vector
}


In [ ]:
def categorize_word(word):
    word_vector = nlp(word).vector

    # Initialize categories
    food = None
    quantity = None
    effectiveness = None
    types_of_effects = None

    # Calculate similarity scores for each category
    food_scores = {category: nlp(word).similarity(nlp(category)) for category in food_vectors}
    quantity_scores = {category: nlp(word).similarity(nlp(category)) for category in quantity_vectors}
    effectiveness_scores = {category: nlp(word).similarity(nlp(category)) for category in effectiveness_vectors}
    types_of_effects_scores = {category: nlp(word).similarity(nlp(category)) for category in types_of_effects_vectors}

    # Assign categories based on the highest similarity score
    if food_scores:
        food = max(food_scores, key=food_scores.get)
    if quantity_scores:
        quantity = max(quantity_scores, key=quantity_scores.get)
    if effectiveness_scores:
        effectiveness = max(effectiveness_scores, key=effectiveness_scores.get)
    if types_of_effects_scores:
        types_of_effects = max(types_of_effects_scores, key=types_of_effects_scores.get)

    return food, quantity, effectiveness, types_of_effects


In [ ]:
words = ["carrot", "dog", "steak", "tiny", "enormous", "effective", "ineffective", "beneficial", "harmful"]

for word in words:
    food, quantity, effectiveness, types_of_effects = categorize_word(word)
    print(f"Word: {word}")
    print(f"Food: {food}")
    print(f"Quantity: {quantity}")
    print(f"Effectiveness: {effectiveness}")
    print(f"Types of Effects: {types_of_effects}")
    print()

Word: carrot
Food: veggie
Quantity: little
Effectiveness: low
Types of Effects: negative

Word: dog
Food: meat
Quantity: little
Effectiveness: low
Types of Effects: positive

Word: steak
Food: meat
Quantity: little
Effectiveness: low
Types of Effects: negative

Word: tiny
Food: meat
Quantity: little
Effectiveness: low
Types of Effects: negative

Word: enormous
Food: meat
Quantity: little
Effectiveness: high
Types of Effects: negative

Word: effective
Food: meat
Quantity: little
Effectiveness: high
Types of Effects: negative

Word: ineffective
Food: meat
Quantity: little
Effectiveness: low
Types of Effects: negative

Word: beneficial
Food: meat
Quantity: little
Effectiveness: high
Types of Effects: negative

Word: harmful
Food: meat
Quantity: little
Effectiveness: low
Types of Effects: negative



Ok, this works a little better.

# Combining the Parsing and the bayes net

In [82]:
import spacy

nlp = spacy.load("en_core_web_lg")

# Define category vectors
active_ingredient_vectors = {
    "charcoal": nlp("charcoal").vector,
    "poppy": nlp("poppy").vector
}

food_vectors = {
    "meat": nlp("meat").vector,
    "veggie": nlp("vegetable").vector
}

quantity_vectors = {
    "little": nlp("small").vector,
    "lot": nlp("large").vector
}

effectiveness_vectors = {
    "low": nlp("low").vector,
    "high": nlp("high").vector
}

types_of_effects_vectors = {
    "negative": nlp("negative").vector,
    "positive": nlp("positive").vector
}

def categorize_word(word):
    word_vector = nlp(word).vector

    # Initialize categories
    active_ingredient = None
    food = None
    quantity = None
    effectiveness = None
    types_of_effects = None

    # Calculate similarity scores for each category
    active_ingredient_scores = {category: nlp(word).similarity(nlp(category)) for category in active_ingredient_vectors}
    food_scores = {category: nlp(word).similarity(nlp(category)) for category in food_vectors}
    quantity_scores = {category: nlp(word).similarity(nlp(category)) for category in quantity_vectors}
    effectiveness_scores = {category: nlp(word).similarity(nlp(category)) for category in effectiveness_vectors}
    types_of_effects_scores = {category: nlp(word).similarity(nlp(category)) for category in types_of_effects_vectors}

    # Assign categories based on the highest similarity score
    if active_ingredient_scores:
        active_ingredient = 0 if max(active_ingredient_scores, key=active_ingredient_scores.get) == "charcoal" else 1
    if food_scores:
        food = 0 if max(food_scores, key=food_scores.get) == "meat" else 1
    if quantity_scores:
        quantity = 0 if max(quantity_scores, key=quantity_scores.get) == "little" else 1
    if effectiveness_scores:
        effectiveness = 0 if max(effectiveness_scores, key=effectiveness_scores.get) == "low" else 1
    if types_of_effects_scores:
        types_of_effects = 0 if max(types_of_effects_scores, key=types_of_effects_scores.get) == "negative" else 1

    return active_ingredient, food, quantity, effectiveness, types_of_effects

def parse_text(text):
    doc = nlp(text)

    # Initialize categories
    active_ingredient = None
    food = None
    quantity = None
    effectiveness = None
    types_of_effects = None

    # Categorize words in the text
    for token in doc:
        if token.pos_ == "NOUN" or token.pos_ == "ADJ":
            word_active_ingredient, word_food, word_quantity, word_effectiveness, word_types_of_effects = categorize_word(token.text)

            if word_active_ingredient is not None:
                active_ingredient = word_active_ingredient
            if word_food is not None:
                food = word_food
            if word_quantity is not None:
                quantity = word_quantity
            if word_effectiveness is not None:
                effectiveness = word_effectiveness
            if word_types_of_effects is not None:
                types_of_effects = word_types_of_effects

    return active_ingredient, food, quantity, effectiveness, types_of_effects

# Example usage
text = "The meal consisted of a small portion of steak with an active ingredient, which had a low effectiveness and negative effects."
active_ingredient, food, quantity, effectiveness, types_of_effects = parse_text(text)

# Format the results for the Bayesian network
observations = {}
if active_ingredient is not None:
    observations['ActiveIng'] = active_ingredient
if food is not None:
    observations['Food'] = food
if quantity is not None:
    observations['Quantity'] = quantity
# if effectiveness is not None:
#     observations['Effectiveness'] = effectiveness
# if types_of_effects is not None:
    # observations['TypesOfEffects'] = types_of_effects

print("Observations for the Bayesian network:")
print(observations)

Observations for the Bayesian network:
{'ActiveIng': 0, 'Food': 0, 'Quantity': 0}


In [94]:
# Perform inference to find the probability of Effectiveness
prob_effectiveness = inference.query(variables=['Effectiveness'], evidence=observations)
print(f"\nProbability of Effectiveness based on the Bayesian network:")
print(prob_effectiveness)

# Perform inference to find the probability of Types of Effects
prob_types_of_effects = inference.query(variables=['TypesOfEffects'], evidence=observations)
print(f"\nProbability of Types of Effects based on the Bayesian network:")
print(prob_types_of_effects)

# Compare the text suggestions with the probabilities
text_effectiveness = 0 if effectiveness == 0 else 1
text_types_of_effects = 0 if types_of_effects == 0 else 1

prob_text_effectiveness = prob_effectiveness.values[text_effectiveness]
prob_text_types_of_effects = prob_types_of_effects.values[text_types_of_effects]

print("\nComparison of text suggestions and probabilities:")
print(f"Effectiveness:")
print(f"  Text suggests: {'Low' if text_effectiveness == 0 else 'High'}")
print(f"  Probability of the text suggestion being true: {prob_text_effectiveness:.2f}")

print(f"\nTypes of Effects:")
print(f"  Text suggests: {'Negative' if text_types_of_effects == 0 else 'Positive'}")
print(f"  Probability of the text suggestion being true: {prob_text_types_of_effects:.2f}")




Probability of Effectiveness based on the Bayesian network:
+------------------+----------------------+
| Effectiveness    |   phi(Effectiveness) |
+==================+======================+
| Effectiveness(0) |               0.0000 |
+------------------+----------------------+
| Effectiveness(1) |               1.0000 |
+------------------+----------------------+

Probability of Types of Effects based on the Bayesian network:
+-------------------+-----------------------+
| TypesOfEffects    |   phi(TypesOfEffects) |
+===================+=======================+
| TypesOfEffects(0) |                1.0000 |
+-------------------+-----------------------+
| TypesOfEffects(1) |                0.0000 |
+-------------------+-----------------------+

Comparison of text suggestions and probabilities:
Effectiveness:
  Text suggests: Low
  Probability of the text suggestion being true: 0.00

Types of Effects:
  Text suggests: Negative
  Probability of the text suggestion being true: 1.00


## Explanations

In [95]:
# Generate explanations
print("\nExplanations:")

# Find the parent nodes of Effectiveness
effectiveness_contributors = model.get_parents('Effectiveness')
print("effectiveness_contributors", effectiveness_contributors)
print("prob_effectiveness", prob_effectiveness)
effectiveness_contributions = []
for var in effectiveness_contributors:
    print("var", var)
    if var in observations:
        prob_with_evidence = inference.query(variables=['Effectiveness'], evidence={var: observations[var]}).values[1]
        print("prob_with_evidence", prob_with_evidence)
        print("inference.query(variables=['Effectiveness'], evidence={var: observations[var]}).values[1]", inference.query(variables=['Effectiveness'], evidence={var: observations[var]}).values[1])
        prob_without_evidence = inference.query(variables=['Effectiveness']).values[1]
        contribution = prob_with_evidence - prob_without_evidence
        effectiveness_contributions.append((var, contribution))

if effectiveness_contributions:
    most_influential_var_eff, most_influential_contrib_eff = max(effectiveness_contributions, key=lambda x: abs(x[1]))
    if most_influential_contrib_eff != 0:
        print(f"The variable that contributes the most to Effectiveness is {most_influential_var_eff} with a change in probability of {most_influential_contrib_eff:.2f}.")
    else:
        print(f"No variables significantly contribute to Effectiveness based on the given observations.")
else:
    print("No variables significantly contribute to Effectiveness based on the given observations.")

# Find the parent nodes of TypesOfEffects
types_of_effects_contributors = model.get_parents('TypesOfEffects')
print("types_of_effects_contributors", types_of_effects_contributors)
types_of_effects_contributions = []
for var in types_of_effects_contributors:
    if var in observations:
        prob_with_evidence = inference.query(variables=['TypesOfEffects'], evidence={var: observations[var]}).values[1]
        prob_without_evidence = inference.query(variables=['TypesOfEffects']).values[1]
        contribution = prob_with_evidence - prob_without_evidence
        types_of_effects_contributions.append((var, contribution))

if types_of_effects_contributions:
    most_influential_var_toe, most_influential_contrib_toe = max(types_of_effects_contributions, key=lambda x: abs(x[1]))
    if most_influential_contrib_toe != 0:
        print(f"The variable that contributes the most to Types of Effects is {most_influential_var_toe} with a change in probability of {most_influential_contrib_toe:.2f}.")
    else:
        print(f"No variables significantly contribute to Types of Effects based on the given observations.")
else:
    print("No variables significantly contribute to Types of Effects based on the given observations.")


Explanations:
effectiveness_contributors ['ActiveIng', 'Food', 'Quantity']
prob_effectiveness +------------------+----------------------+
| Effectiveness    |   phi(Effectiveness) |
+==================+======================+
| Effectiveness(0) |               0.0000 |
+------------------+----------------------+
| Effectiveness(1) |               1.0000 |
+------------------+----------------------+
var ActiveIng
prob_with_evidence 0.8
inference.query(variables=['Effectiveness'], evidence={var: observations[var]}).values[1] 0.8
var Food
prob_with_evidence 0.8
inference.query(variables=['Effectiveness'], evidence={var: observations[var]}).values[1] 0.8
var Quantity
prob_with_evidence 0.6
inference.query(variables=['Effectiveness'], evidence={var: observations[var]}).values[1] 0.6
The variable that contributes the most to Effectiveness is ActiveIng with a change in probability of 0.20.
types_of_effects_contributors ['Effectiveness', 'ActiveIng', 'Food']
The variable that contributes the 